In [14]:
import urllib.request
from bs4 import BeautifulSoup
import re
import time
import json


re_year = re.compile(r'\d{4}')

In [2]:
url = "https://www.whosampled.com/The-Winstons/Amen,-Brother/sampled/"
resp = urllib.request.urlopen(url)
page = resp.read()

In [3]:
soup = BeautifulSoup(page, 'html.parser')
list_of_songs = soup.findAll('div', {'class':'trackDetails'})


In [4]:
song_info = []

In [5]:
def extract_song_info(song):
    # artist has 'by artist_name (year)' so split to get the relevant info
    artist_year = song.find('span', {'class':'trackArtist'}).text
    if 'feat.' in artist_year:
        primary_artist = artist_year.split('feat.')[0].strip()[3:]
    else:
        primary_artist = ' '.join(artist_year.split()[1:-1])
    year = re.findall(re_year, artist_year.split()[-1])[0]
    
    title = song.find('a').text.strip()
    instrument = song.find('span', {'class':'topItem'}).text.strip()
    genre = song.find('span', {'class':'bottomItem'}).text.strip()
    
    return primary_artist, title, year, instrument, genre

In [6]:
scraping = True
num_pages = 1
while scraping:
    # get list of songs on current page
    list_of_songs = soup.findAll('div', {'class':'trackDetails'})
    
    for song in list_of_songs:
        song_info.append(extract_song_info(song))
    
    # see if we can go to another page or stop
    next_page = soup.find('span', {'class':'next'})
    if next_page:
        if num_pages % 25 == 0:
            print(f'finished page: {num_pages}. Sleep for 10 seconds')
            time.delay(10)
        num_pages += 1
            
        next_url = "https://www.whosampled.com/" + next_page.a['href']
        soup = BeautifulSoup(urllib.request.urlopen(next_url).read(), 'html.parser')
    else:
        scraping = False

In [7]:
print(f"Total number of pages scraped: {num_pages}")
print(f'Total number of songs: {len(song_info)}')

Total number of pages scraped: 1
Total number of songs: 2864


In [15]:
with open("song_info.txt", 'w') as f:
    f.write(json.dumps(song_info))

scrape_sample_songs.ipynb song_info.txt
